In [3]:
import pandas as pd

abstracts = pd.read_csv('../../data/abstracts.txt', sep=r'\|\-\-\|', header=None, index_col=False, engine='python')
abstracts.columns = ['paper_id', 'abstracts']
abstracts['abstracts'] = abstracts['abstracts'].fillna('')
positive_edges = pd.read_csv('../../data/edgelist.txt', sep=',', header=None)
positive_edges.columns = ['source', 'target']

positive_edges['source_abstract'] = positive_edges['source'].map(abstracts.set_index('paper_id')['abstracts'])
positive_edges['target_abstract'] = positive_edges['target'].map(abstracts.set_index('paper_id')['abstracts'])

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_text)
positive_edges['source_abstract'] = positive_edges['source_abstract'].apply(remove_stopwords)
positive_edges['target_abstract'] = positive_edges['target_abstract'].apply(remove_stopwords)
#calculate jaccard similarity

def jaccard_similarity(str1, str2):
    # Convert strings to sets of words
    set1 = set(str1.split())
    set2 = set(str2.split())

    # Calculate Jaccard similarity
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))

    if union == 0:
        return 0.0

    return intersection / union


positive_edges['jaccard_similarity'] = positive_edges.apply(
    lambda row: jaccard_similarity(row['source_abstract'], row['target_abstract']), axis=1
)
positive_edges = positive_edges[['source', 'target', 'jaccard_similarity']]
positive_edges.to_csv('../../data/training/positive/jaccard_similarity.csv', index=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
